In [ ]:
import holoviews as hv
import numpy as np
import panel as pn

from holoviews.annotators import annotate

hv.extension('bokeh')

It is often important to augment, edit and annotate datasets to provide semantic information, to aid further processing or simply to make the data easier to interpret. To perform these actions HoloViews provides an ``annotate`` helper which makes it easy to edit some element types and add additional information using a table. The `annotate` helper:

* Adds plot tools which allow editing and adding new elements to a plot  
* Adds table(s) to allow editing the element in a tabular format
* Returns a layout of these two components
* Makes the edits, annotations and selections available on a property

## Basics

Let us start by annotating a simple set of Points, we can do this in three steps:

1. Create the element to annotate/edit
2. Create an instance of the `annotate` helper (this will allow us to access the edited/annotated data)
3. Call the instance with the element to annotate and any parameters, such as the names of any annotations to add.

The returned object is a Layout consisting of the object to be annotated and an Overlay of the table(s) used to edit the data:

In [ ]:
points = hv.Points([(0, 0), (1, 1), (2, 2)]).opts(height=400)

annotator = annotate.instance() 

layout = annotator(points, annotations=['Label'])

print(layout)

layout

You will note that the toolbar contains a [PointDraw tool](../reference/streams/PointDraw.ipynb) which allows us to drag and add points and the table contains the additional 'Label' column we requested. Additionally the plot and table are linked so a change in one will immediately appear in the other.

Making an explicit instance of the `annotate` function allows to access the annotated and edited data on the ``annotated`` attribute:

In [ ]:
annotator.annotated.dframe()

Separately we can also access the currently `selected` points in case we care only about a subset of the points.

In [ ]:
annotator.selected.dframe()

## Types of Annotators

Currently only a limited set of Elements may be annotated, these include:

* Points
* Path
* Polygon

Further annotators will be added in future releases.

### Annotating paths/polygons

Unlike the Points annotator the Path and Polygon annotators allow annotating not just each individual entity but also the vertices that make up the paths and polygons. For more information about using the editing tools associated with this annotator refer to the [PolyDraw](../reference/streams/PolyDraw.ipynb) and [PolyEdit](../reference/streams/PolyEdit.ipynb) reference guides. To edit and annotate the vertices use the draw tool to select a particular path/polygon and then navigate to the Vertices tab.

In [ ]:
path = hv.Path([hv.Box(0, 0, 1), hv.Ellipse(1, 1, 1)])

path_annotator = annotate.instance()

path_annotator(path, annotations=['Label'], vertex_annotations=['Value'])

To access this data we can make use of the `split` method on Path objects to access each path individually allowing us to look at the values:

In [ ]:
el1, el2 = path_annotator.annotated.split()[:2]

el1.dframe()

## Composing Annotators

Composing the layouts returned by the `annotate` helper is not entirely straightforward so the `annotate` helper also provides a classmethod that allows composing multiple annotators and other elements, e.g. such as a set of tiles into a combined layout consisting of all the components.

In [ ]:
point_annotator = annotate.instance()
point_layout = point_annotator(hv.Points([(500000, 500000), (1000000, 1000000)]).opts(tools=['hover']), annotations=['Label'])

poly_annotator = annotate.instance()
path_layout = path_annotator(hv.Polygons([]), annotations=['Label'])

annotate.compose(hv.element.tiles.Wikipedia(), point_layout, path_layout)